# Model Testing


## Setup


In [1]:
from scripts.data_loader import load_data, split_features_targets
from scripts.preprocessing import preprocess_data, derive_features
from scripts.model_testing import test_independant_models, find_top_models, train_and_predict, test_dependent_models, split_data_by_year

from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
)
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [2]:
df = load_data("data")
df = preprocess_data(df)
df = derive_features(df)

features, targets = split_features_targets(df)

# Target-Independant Testing


### Models


In [ ]:
models = [
    RandomForestRegressor(n_estimators=100, max_depth=10, n_jobs=-1),
    GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1),
    ExtraTreesRegressor(n_estimators=100, max_depth=10, n_jobs=-1),
    BaggingRegressor(n_estimators=10, max_samples=0.5, n_jobs=-1),
    XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, n_jobs=-1),
    LGBMRegressor(
        n_estimators=100, max_depth=5, learning_rate=0.1, n_jobs=-1, verbose=-1
    ),
]

### Results


In [ ]:
# Ice Thickness
find_top_models(test_independant_models(models, features, targets["ice_thickness"])).head()

In [ ]:
# Ice Velocity
find_top_models(test_independant_models(models, features, targets["ice_velocity"])).head()

In [ ]:
# Ice Mask
find_top_models(test_independant_models(models, features, targets["ice_mask"])).head()

## Further Testing


### Models


In [ ]:
MAX_DEPTH = 60
N_ESTIMATORS = 160
N_JOBS = -1

models = [
    RandomForestRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, n_jobs=N_JOBS),
    GradientBoostingRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, learning_rate=0.1),
    ExtraTreesRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, n_jobs=N_JOBS),
    BaggingRegressor(n_estimators=N_ESTIMATORS, max_samples=0.5, n_jobs=N_JOBS),
    XGBRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, learning_rate=0.1, n_jobs=N_JOBS),
    LGBMRegressor(
        n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, learning_rate=0.1, n_jobs=N_JOBS, verbose=-1
    ),
]

### Results


In [ ]:
results = test_independant_models(models, features, targets["ice_velocity"], split_year=2085)
results.head(6)

# Target-Dependant Testing


## Initial Testing


In [4]:
import time

thickness_model = BaggingRegressor(n_estimators=10, max_samples=0.5, n_jobs=-1)
mask_model = BaggingRegressor(n_estimators=10, max_samples=0.5, n_jobs=-1)
velocity_model = ExtraTreesRegressor(n_estimators=100, max_depth=10, n_jobs=-1)

models = [thickness_model, mask_model, velocity_model]

start_time = time.time()
results = test_dependent_models(models, features, targets, target_order=['ice_thickness', 'ice_mask', 'ice_velocity'], split_year=2050)
end_time = time.time()

print(f"Total execution time: {end_time - start_time:.2f} seconds")
print(results)

                      MSE       MAE        R2
ice_thickness    0.001117  0.011774  0.982508
ice_mask         0.017728  0.019543  0.981190
ice_velocity   564.818300  5.254401  0.432202


## Order Testing


In [ ]:
order1 = ['ice_thickness', 'ice_mask', 'ice_velocity']
order2 = ['ice_mask', 'ice_thickness', 'ice_velocity']
order3 = ['ice_velocity', 'ice_thickness', 'ice_mask']

# Test the dependent models with different orderings
results1 = test_dependent_models([thickness_model, mask_model, velocity_model], features, targets, order1, split_year=2050)
print("Order 1 results:")
print(results1)

results2 = test_dependent_models([thickness_model, mask_model, velocity_model], features, targets, order2, split_year=2050)
print("\nOrder 2 results:")
print(results2)

results3 = test_dependent_models([thickness_model, mask_model, velocity_model], features, targets, order3, split_year=2050)
print("\nOrder 3 results:")
print(results3)

## Further Testing
